In [1]:
import redivis
import numpy as np
import geopandas as gpd

## Worflow: 

- Step 1: Define the boudnary of the Central Valley
- Step 2: Download sediment type data
- Step 3: Download State-wide AEM resistivity data 
- Step 4: Download Water level data

In [2]:
org_em_recharge = redivis.organization("KnightAccelerator")
em_dataset = org_em_recharge.dataset("em_data")
sediment_type_dataset = org_em_recharge.dataset("sediment_type_data")
waterlevel_dataset = org_em_recharge.dataset("waterlevel_data")
ca_geo_dataset = org_em_recharge.dataset("California State Geo Features")

In [3]:
target_path = "./"

In [9]:
gdf_cv = gpd.read_file("../data/shp/cv.shp")
geom = gdf_cv.geometry[0]

from shapely.geometry import Polygon

USER_DEFINED_POLYGON = gdf_cv.geometry[0]
xx, yy = USER_DEFINED_POLYGON.exterior.coords.xy

# Create polygon from lists of points
x = xx[::1].tolist()
y = yy[::1].tolist()

USER_DEFINED_POLYGON = Polygon(zip(x,y))

In [12]:
# Then draw a polygon
q = ca_geo_dataset.query(f"""
    SELECT geometry, Basin_Name, Basin_Su_1, Basin_Subb
    FROM basins L
""")
gdf_geom = q.to_dataframe()
gdf = gpd.GeoDataFrame(data=gdf_geom[['Basin_Name', 'Basin_Su_1','Basin_Subb']], geometry=gdf_geom.geometry, crs=4326)
gdf.to_file(f"{target_path}/basins.geojson")

  0%|          | 0/515 [00:00<?, ?it/s]

In [36]:
# Then draw a polygon
q = ca_geo_dataset.query(f"""
    SELECT geometry, MTRS
    FROM plss L
    WHERE ST_Contains(ST_GeogFromText("{USER_DEFINED_POLYGON}"),L.geometry)
""")
gdf_geom = q.to_dataframe()
gdf = gpd.GeoDataFrame(data=gdf_geom['MTRS'], geometry=gdf_geom.geometry, crs=4326)
gdf.to_file(f"{target_path}/plss.geojson")

  0%|          | 0/19172 [00:00<?, ?it/s]

In [50]:
# Then draw a polygon
SURVEY_ID = "cdwr_sacv"

q = em_dataset.query(f"""
    SELECT RECORD, ELEVATION, UTMX, UTMY, LINE_NO, SURVEY_ID, DOI_STANDARD, RESDATA, MEASUREMENTS
    FROM SurveyLocation L
    WHERE SURVEY_ID="{SURVEY_ID}" AND ST_Contains(ST_GeogFromText("{USER_DEFINED_POLYGON}"),L.geometry)
""")
df_em = q.to_dataframe()
df_em.to_csv(f"{target_path}/em_resistivity.csv", index=False)

print(f"Completed: {len(df_em.index)} rows downloaded.")

  0%|          | 0/376761 [00:00<?, ?it/s]

Completed: 376761 rows downloaded.


In [51]:
print("Loading... Layer Thickness to the compute server")
# TODO: add a season in the Survey table
q = em_dataset.query(f"""
    SELECT THICKNESS, KIND, YEAR, SEASON
    FROM Survey
    WHERE ID="{SURVEY_ID}" AND GEOTYPE="BOUNDS"
""")
df_thickness = q.to_dataframe()
df_thickness.to_csv(f"{target_path}/thickness.csv", index=False)

print(f"Completed.")

Loading... Layer Thickness to the compute server


  0%|          | 0/1 [00:00<?, ?it/s]

Completed.


In [52]:
year = df_thickness['YEAR'].values[0]
season = df_thickness['SEASON'].values[0].capitalize()

In [53]:
sediment_type_path = f"{target_path}/sediment_type.csv"

In [54]:
print("Loading... Public Sediment / Rock Type to the compute server")
q = sediment_type_dataset.query(f"""
    SELECT WELL_NAME, UTMX, UTMY, WELL_ID, KIND, MEASUREMENTS, METADATA
    FROM SedimentTypeLocation L
    WHERE ST_Contains(ST_GeogFromText("{USER_DEFINED_POLYGON}"),L.geometry)
""")
df_sediment_type = q.to_dataframe()
df_sediment_type.to_csv(sediment_type_path, index=False)
print(f"Completed: {len(df_sediment_type.index)} records downloaded.")

Loading... Public Sediment / Rock Type to the compute server


  0%|          | 0/36488 [00:00<?, ?it/s]

Completed: 36488 records downloaded.


In [55]:
q = waterlevel_dataset.query(f"""
    SELECT WELL_NAME, UTMX, UTMY, WELL_ID, KIND, MEASUREMENTS
    FROM WaterlevelLocation L
    WHERE ST_Contains(ST_GeogFromText("{USER_DEFINED_POLYGON}"),L.geometry)
""")
df_waterlevel = q.to_dataframe()
df_waterlevel.to_csv(f"{target_path}/waterlevel.csv", index=False)

print(f"Completed: {len(df_waterlevel.index)} records downloaded.")

  0%|          | 0/12350 [00:00<?, ?it/s]

Completed: 12350 records downloaded.


In [56]:
q = waterlevel_dataset.query(f"""
    SELECT LOCATION_ID, UTMX, UTMY, GSE_WSE, SEASON, YEAR
    FROM WaterlevelContourLocation L
    WHERE YEAR={year} AND SEASON="{season}"
""")
df_waterlevel_contour = q.to_dataframe()
df_waterlevel_contour.to_csv(f"{target_path}/waterlevel_contour.csv", index=False)

print(f"Completed: {year} {season} data downloaded.")

  0%|          | 0/21126 [00:00<?, ?it/s]

Completed: 2022 Spring data downloaded.


In [4]:
print("Loading... Bedrock Surface to a computer server")
# TODO: add a season in the Survey table
q = em_dataset.query(f"""
    SELECT BEDROCK_SURFACE_DEPTH, BEDROCK_SURFACE_ELEVATION, UTMX, UTMY
    FROM BedrockSurface
""")
df_bedrock = q.to_dataframe()
df_bedrock.to_csv(f"{target_path}/bedrock_surface.csv", index=False)

print(f"Completed.")

Loading... Bedrock Surface to a computer server


  0%|          | 0/32440 [00:00<?, ?it/s]

Completed.
